In [5]:
import matplotlib.pyplot as plt
import numpy as np
import polars as pl

import jax
import numpyro
import numpyro.distributions as dist
from numpyro.infer import Predictive, SVI, Trace_ELBO, init_to_value
from numpyro.infer.autoguide import AutoLaplaceApproximation
import numpyro.optim as optim

import util

# 5.1 Spurious association

In [7]:
wd = pl.read_csv("data/WaffleDivorce.csv", sep=";")
print(util.summarize(wd))

pl.DataFrame of shape (50, 13)

╒═══════════════════╤═════════╤════════════════╤═══════════════╤══════════╤══════════════════╤══════════════════╕
│ column            │ dtype   │           mean │           std │     5.5% │            94.5% │ histogram        │
╞═══════════════════╪═════════╪════════════════╪═══════════════╪══════════╪══════════════════╪══════════════════╡
│ Location          │ str     │                │               │          │                  │ ▂▂▂▅█▂▂▁▂▂▁      │
├───────────────────┼─────────┼────────────────┼───────────────┼──────────┼──────────────────┼──────────────────┤
│ Loc               │ str     │                │               │          │                  │         █        │
├───────────────────┼─────────┼────────────────┼───────────────┼──────────┼──────────────────┼──────────────────┤
│ Population        │ f64     │      6.1196    │      6.87616  │  0.6578  │     18.9769      │ █▄▄▁             │
├───────────────────┼─────────┼────────────────┼────────

In [20]:
means = {}
stds = {}
for k in ['Marriage', 'MedianAgeMarriage', 'Divorce']:
    means[k] = wd[k].mean()
    stds[k] = wd[k].std()
    
def standard(k, v):
    return (v - means[k]) / stds[k]

def model(M, A, D=None):
    a = numpyro.sample("a", dist.Normal(0, 0.2))
    bM = numpyro.sample("bM", dist.Normal(0, 0.5))
    bA = numpyro.sample("bA", dist.Normal(0, 0.5))
    sigma = numpyro.sample("sigma", dist.Exponential(1))
    mu = numpyro.deterministic("mu", a + bM * standard('Marriage', M) + bA * standard('MedianAgeMarriage', A))
    numpyro.sample("D", dist.Normal(mu, sigma), obs=standard('Divorce', D))


m5_3 = AutoLaplaceApproximation(model)
svi = SVI(
    model, m5_3, optim.Adam(1), Trace_ELBO(), M=wd['Marriage'].to_numpy(), 
    A=wd['MedianAgeMarriage'].to_numpy(), D=wd['Divorce'].to_numpy()
)
svi_result = svi.run(jax.random.PRNGKey(0), 1000)
p5_3 = svi_result.params
post = m5_3.sample_posterior(jax.random.PRNGKey(1), p5_3, (1000,))
post.pop('mu')
post = pl.DataFrame({k: np.array(v) for k, v in post.items()})
print(util.summarize(post))

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 2326.15it/s, init loss: 3201.7393, avg. loss [951-1000]: 60.7879]


pl.DataFrame of shape (1000, 4)

╒══════════╤═════════╤═════════════╤═══════════╤═══════════╤═══════════╤══════════════════╕
│ column   │ dtype   │        mean │       std │      5.5% │     94.5% │ histogram        │
╞══════════╪═════════╪═════════════╪═══════════╪═══════════╪═══════════╪══════════════════╡
│ a        │ f32     │ -0.00171026 │ 0.0960137 │ -0.154456 │  0.143987 │   ▁▂▄▅▇█▆▆▄▂     │
├──────────┼─────────┼─────────────┼───────────┼───────────┼───────────┼──────────────────┤
│ bA       │ f32     │ -0.607178   │ 0.160442  │ -0.857274 │ -0.350195 │    ▁▂▄▅▇█▇▆▄▂▁   │
├──────────┼─────────┼─────────────┼───────────┼───────────┼───────────┼──────────────────┤
│ bM       │ f32     │ -0.0591115  │ 0.155485  │ -0.308416 │  0.188886 │    ▁▂▄▅▇█▇▅▃▁    │
├──────────┼─────────┼─────────────┼───────────┼───────────┼───────────┼──────────────────┤
│ sigma    │ f32     │  0.795513   │ 0.0770277 │  0.676315 │  0.922754 │   ▂▃▅▆▇█▆▄▂▁     │
╘══════════╧═════════╧═════════════╧═══════════